In [ ]:
from shutil import get_terminal_size
from collections import deque
import time
from itertools import count
import math
from tabulate import tabulate
import sys
import numpy as np
import random
from collections import defaultdict

**Code to visualize the problem**


In [27]:
terminal_width, _ = get_terminal_size()
_visualizers = {}

def _default_visualizer(_, state):
    print(state)

class Visualizer:
    def __init__(self, problem):
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

    def visualize_state(self, state=None):
        if state is None:
            state = self.problem.init_state
        self.counter += 1
        print(f'Visualizing state at step {self.counter}')
        _visualizers.get(type(self.problem), _default_visualizer)(self.problem, state)
        print('-' * terminal_width)


**The problem class Robot_path**

In [28]:
class Robot_Path():
      def __init__(self, init_state, goal_state, grid_size, obstacles=None):
        #assert is a function to make sure that entered initial and goal states are not outside of grid
        assert 0 <= init_state[0] < grid_size[0] and 0 <= init_state[1] < grid_size[1]
        assert 0 <= goal_state[0] < grid_size[0] and 0 <= goal_state[1] < grid_size[1]
        self._grid_size = grid_size
        self.init_state = init_state
        self._goal_state = goal_state
        self.obstacles = set(obstacles) if obstacles else set()
        self._action_values = {'up': (-1,0), 'down': (+1,0), 'left': (0,-1), 'right': (0,+1)}
        self.goal_reward = 100
        self.step_cost = -1
        self.obstacle_penalty = -100

      def actions(self, state):
        x,y = state
        possible_moves = []
        for action, (dx, dy) in self._action_values.items():
            next_state = (x + dx, y + dy)
            if (0 <= next_state[0] < self._grid_size[0] and
                0 <= next_state[1] < self._grid_size[1] and
                next_state not in self.obstacles):
                possible_moves.append(action)
        return possible_moves

      def result(self, state, action):
        """Apply an action and return the resulting state."""
        x, y = state
        dx, dy = self._action_values[action]
        next_state = (x + dx, y + dy)
        if next_state not in self.obstacles and 0 <= next_state[0] < self._grid_size[0] and 0 <= next_state[1] < self._grid_size[1]:
            return next_state
        return state

      def goal_test(self, state):
        return state == self._goal_state

      def reward(self, state, action):
        """Return the reward for taking an action in a state."""
        next_state = self.result(state, action)
        if next_state == self._goal_state:
            return self.goal_reward
        elif next_state in self.obstacles:
            return self.obstacle_penalty
        return self.step_cost

def visualize_grid(problem, state):
    """Visualize the grid environment."""
    grid = [['.' for _ in range(problem._grid_size[1])] for _ in range(problem._grid_size[0])]
    for ox, oy in problem.obstacles:
        grid[ox][oy] = 'X'
    goal_x, goal_y = problem._goal_state
    grid[goal_x][goal_y] = 'G'
    state_x, state_y = state
    grid[state_x][state_y] = 'A'
    for row in grid:
        print(' '.join(row))
_visualizers[Robot_Path] = visualize_grid

**Generate problem**

In [35]:
n = 5
obstacles = [(1, 1), (2, 2), (3, 3)]
problem = Robot_Path((0,0),(n-1,n-1), (n,n), obstacles)
visualize_grid(problem, problem.init_state)

A . . . .
. X . . .
. . X . .
. . . X .
. . . . G


In [36]:
print("Valid Actions from Initial State:", problem.actions(problem.init_state))
next_state = problem.result(problem.init_state, 'right')
print("Next State after moving 'right':", next_state)
reward = problem.reward(problem.init_state, 'right')
print("Reward for moving 'right':", reward)

Valid Actions from Initial State: ['down', 'right']
Next State after moving 'right': (0, 1)
Reward for moving 'right': -1


In [37]:
problem.current_state = problem.result(problem.init_state, 'right')
visualize_grid(problem, problem.current_state)

. A . . .
. X . . .
. . X . .
. . . X .
. . . . G


In [38]:
class QLearningAgent:
    def __init__(self, problem, alpha=0.01, gamma=0.99, epsilon=0.5, Ne=5, Rplus=2):
        self.problem = problem
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.Ne = Ne
        self.Rplus = Rplus
        self.Q = defaultdict(float)
        self.Nsa = defaultdict(int)
        self.state = problem.init_state
        self.action = None
        self.reward = 0
#exploration function
    def f(self, u, n):

        return self.Rplus if n < self.Ne else u

    def select_action(self, state):
        actions = self.problem.actions(state)
        if random.random() < self.epsilon:
            return random.choice(actions)
        return max(actions, key=lambda a: self.f(self.Q[(state, a)], self.Nsa[(state, a)]))

    def update(self, state, action, reward, next_state):
        self.Nsa[(state, action)] += 1
        best_next_q = max([self.Q[(next_state, a)] for a in self.problem.actions(next_state)], default=0)
        self.Q[(state, action)] += self.alpha * (reward + self.gamma * best_next_q - self.Q[(state, action)])


    def train(self, episodes=100000):
        for episode in range(episodes):
            state = self.problem.init_state
            self.epsilon = max(0.1, self.epsilon * 0.995)
            while not self.problem.goal_test(state):
                action = self.select_action(state)
                next_state = self.problem.result(state, action)
                reward = self.problem.reward(state, action)
                self.update(state, action, reward, next_state)
                state = next_state

            # midway monitoring
            if episode == episodes // 2:
                print("\n Midway Q-Values:")
                for (state, action), value in self.Q.items():
                    print(f"State: {state}, Action: {action}, Q-Value: {value:.2f}")
                print("\n Midway Policy:")
                midway_policy = self.get_policy()
                for state, action in midway_policy.items():
                    print(f"State: {state}, Action: {action}")

    def get_policy(self):
        policy = {}
        # iterate through possible states in the grid
        for x in range(self.problem._grid_size[0]):
            for y in range(self.problem._grid_size[1]):
                state = (x, y)
                actions = self.problem.actions(state)
                if actions:
                    policy[state] = max(actions, key=lambda a: self.Q[(state, a)])
        return policy

In [39]:
# train Q-Learning Agent
q_agent = QLearningAgent(problem)
q_agent.train(episodes=100000)


 Midway Q-Values:
State: (0, 0), Action: down, Q-Value: 86.41
State: (0, 0), Action: right, Q-Value: 82.70
State: (1, 0), Action: up, Q-Value: 84.55
State: (1, 0), Action: down, Q-Value: 88.30
State: (0, 1), Action: left, Q-Value: 84.55
State: (0, 1), Action: right, Q-Value: 6.56
State: (0, 2), Action: down, Q-Value: -0.87
State: (0, 2), Action: left, Q-Value: 2.97
State: (0, 2), Action: right, Q-Value: 16.44
State: (1, 2), Action: up, Q-Value: -0.63
State: (1, 2), Action: right, Q-Value: 4.24
State: (1, 3), Action: up, Q-Value: 0.20
State: (1, 3), Action: down, Q-Value: -0.07
State: (1, 3), Action: left, Q-Value: -0.34
State: (1, 3), Action: right, Q-Value: 53.36
State: (2, 3), Action: up, Q-Value: -0.05
State: (2, 3), Action: right, Q-Value: 7.77
State: (0, 3), Action: down, Q-Value: 32.27
State: (0, 3), Action: left, Q-Value: -0.51
State: (0, 3), Action: right, Q-Value: -0.18
State: (0, 4), Action: down, Q-Value: 9.80
State: (0, 4), Action: left, Q-Value: -0.08
State: (1, 4), Actio

In [40]:
print("Q-Values:")
for (state, action), value in q_agent.Q.items():
    print(f"State: {state}, Action: {action}, Q-Value: {value:.2f}")

Q-Values:
State: (0, 0), Action: down, Q-Value: 86.41
State: (0, 0), Action: right, Q-Value: 82.70
State: (1, 0), Action: up, Q-Value: 84.55
State: (1, 0), Action: down, Q-Value: 88.30
State: (0, 1), Action: left, Q-Value: 84.55
State: (0, 1), Action: right, Q-Value: 22.51
State: (0, 2), Action: down, Q-Value: -0.67
State: (0, 2), Action: left, Q-Value: 6.87
State: (0, 2), Action: right, Q-Value: 37.09
State: (1, 2), Action: up, Q-Value: -0.63
State: (1, 2), Action: right, Q-Value: 7.28
State: (1, 3), Action: up, Q-Value: 1.42
State: (1, 3), Action: down, Q-Value: 0.48
State: (1, 3), Action: left, Q-Value: -0.30
State: (1, 3), Action: right, Q-Value: 72.91
State: (2, 3), Action: up, Q-Value: 0.57
State: (2, 3), Action: right, Q-Value: 17.18
State: (0, 3), Action: down, Q-Value: 54.60
State: (0, 3), Action: left, Q-Value: 0.26
State: (0, 3), Action: right, Q-Value: 1.13
State: (0, 4), Action: down, Q-Value: 18.25
State: (0, 4), Action: left, Q-Value: -0.08
State: (1, 4), Action: up, Q-V

In [41]:
# Policy
policy = q_agent.get_policy()
print("Policy:")
for state, action in policy.items():
    print(f"State: {state}, Action: {action}")

Policy:
State: (0, 0), Action: down
State: (0, 1), Action: left
State: (0, 2), Action: right
State: (0, 3), Action: down
State: (0, 4), Action: down
State: (1, 0), Action: down
State: (1, 1), Action: up
State: (1, 2), Action: right
State: (1, 3), Action: right
State: (1, 4), Action: down
State: (2, 0), Action: down
State: (2, 1), Action: down
State: (2, 2), Action: up
State: (2, 3), Action: right
State: (2, 4), Action: down
State: (3, 0), Action: right
State: (3, 1), Action: down
State: (3, 2), Action: down
State: (3, 3), Action: up
State: (3, 4), Action: down
State: (4, 0), Action: right
State: (4, 1), Action: right
State: (4, 2), Action: right
State: (4, 3), Action: right
State: (4, 4), Action: up


In [42]:
def visualize_q_policy(agent, problem):
    grid = [['.' for _ in range(problem._grid_size[1])] for _ in range(problem._grid_size[0])]
    for state, action in agent.get_policy().items():
        if action == 'up':
            symbol = '^'
        elif action == 'down':
            symbol = 'v'
        elif action == 'left':
            symbol = '<'
        elif action == 'right':
            symbol = '>'
        else:
            symbol = '.'
        grid[state[0]][state[1]] = symbol
    for row in grid:
        print(' '.join(row))

visualize_q_policy(q_agent, problem)

v < > v v
v ^ > > v
v v ^ > v
> v v ^ v
> > > > ^


In [43]:
# calculate utilities from Q-Values
U = defaultdict(lambda: -1000.)
for (state, action), value in q_agent.Q.items():
    if U[state] < value:
        U[state] = value

print(" Final State Utilities:")
for state, utility in U.items():
    print(f"State: {state}, Utility: {utility:.2f}")

 Final State Utilities:
State: (0, 0), Utility: 86.41
State: (1, 0), Utility: 88.30
State: (0, 1), Utility: 84.55
State: (0, 2), Utility: 37.09
State: (1, 2), Utility: 7.28
State: (1, 3), Utility: 72.91
State: (2, 3), Utility: 17.18
State: (0, 3), Utility: 54.60
State: (0, 4), Utility: 18.25
State: (1, 4), Utility: 86.94
State: (2, 4), Utility: 95.32
State: (3, 4), Utility: 99.51
State: (2, 0), Utility: 90.20
State: (2, 1), Utility: 92.12
State: (3, 1), Utility: 94.06
State: (3, 0), Utility: 92.12
State: (4, 1), Utility: 96.02
State: (4, 4), Utility: 0.00
State: (4, 0), Utility: 94.06
State: (3, 2), Utility: 96.02
State: (4, 2), Utility: 98.00
State: (4, 3), Utility: 100.00
State: (1, 1), Utility: 0.00
State: (2, 2), Utility: 0.00
State: (3, 3), Utility: 0.00


In [45]:
def visualize_utilities(problem, U):
    grid = [['-' for _ in range(problem._grid_size[1])] for _ in range(problem._grid_size[0])]
    for state, utility in U.items():
        grid[state[0]][state[1]] = f"{utility:.1f}"
    for row in grid:
        print(' '.join(row))

visualize_utilities(problem, U)

86.4 84.5 37.1 54.6 18.3
88.3 0.0 7.3 72.9 86.9
90.2 92.1 0.0 17.2 95.3
92.1 94.1 96.0 0.0 99.5
94.1 96.0 98.0 100.0 0.0


In [46]:
def visualize_agent_path(problem, agent):
    state = problem.init_state
    path = [state]
    while not problem.goal_test(state):
        action = agent.select_action(state)
        state = problem.result(state, action)
        path.append(state)
    print("Agent Movement Path:")
    print(" -> ".join(map(str, path)))

visualize_agent_path(problem, q_agent)


Agent Movement Path:
(0, 0) -> (1, 0) -> (2, 0) -> (3, 0) -> (3, 1) -> (4, 1) -> (4, 2) -> (4, 3) -> (4, 4)
